In [37]:
# Use conda/analysis3-24.07 env
from polar_convert.constants import SOUTH
import netCDF4 as nc
import numpy as np

In [38]:
from polar_convert import polar_xy_to_lonlat

In [39]:
true_scale_lat = -70  # true-scale latitude in degrees
re = 6378.137  # earth radius in km
e = 0.01671 # earth eccentricity
hemisphere = 'south'

In [40]:
# Open the NetCDF file
dataset = nc.Dataset('/g/data/tm70/ek4684/Bedmachine_data/BedMachineAntarctica_2020-07-15_v02.nc', 'r')
# Extract the latitude and longitude arrays
y = dataset.variables['y'][:]/100.0  # X array (Convert meters to KMs)
x = dataset.variables['x'][:]/100.0  # Y array (Convert meters to KMs)

In [41]:
# Initialize lists to store new latitudes and longitudes
new_latitudes = []
new_longitudes = []
for i in range(len(x)):
    # Call the function
    lon_result, lat_result = polar_xy_to_lonlat(x[i], y[i], true_scale_lat, re, e, hemisphere)
    lon_result = (lon_result + 180) % 360 - 180
    # Append the results to the lists
    new_longitudes.append(lon_result)
    new_latitudes.append(lat_result)

In [42]:
# Convert the lists to numpy arrays for writing to the new NetCDF file
new_latitudes = np.array(new_latitudes)
new_longitudes = np.array(new_longitudes)

In [43]:
# Create a new NetCDF file to save the updated data
new_file_path = '/g/data/tm70/ek4684/Bedmachine_data/Updated_BedMachineAntarctica_2020-07-15_v02.nc'
new_dataset = nc.Dataset(new_file_path, 'w', format='NETCDF4')

# Copy dimensions from the original dataset to the new file
for name, dimension in dataset.dimensions.items():
    new_dataset.createDimension(name, len(dimension) if not dimension.isunlimited() else None)

# Copy variables (other than the ones being modified) from the original dataset to the new file
for name, variable in dataset.variables.items():
    if name not in ['x', 'y']:  # Don't copy the x and y variables, we'll create new ones
        new_var = new_dataset.createVariable(name, variable.datatype, variable.dimensions)
        new_var[:] = variable[:]
        # Copy all attributes except the fill value
        for attr_name in variable.ncattrs():
            if attr_name != '_FillValue':  # Skip the fill value attribute
                new_var.setncattr(attr_name, variable.getncattr(attr_name))

# Create new variables for lat and lon
lat_var = new_dataset.createVariable('lat', 'f4', ('x',))  # Latitude (lat)
lon_var = new_dataset.createVariable('lon', 'f4', ('x',))  # Longitude (lon)

# Assign the computed latitudes and longitudes to the new variables
lat_var[:] = new_latitudes
lon_var[:] = new_longitudes

# Set attributes for the new variables (optional)
lat_var.units = "degrees_north"
lon_var.units = "degrees_east"

# Close the new dataset
new_dataset.close()

# Close the original dataset
dataset.close()

print(f"New file saved as: {new_file_path}")

New file saved as: /g/data/tm70/ek4684/Bedmachine_data/Updated_BedMachineAntarctica_2020-07-15_v02.nc


In [45]:
print(new_longitudes)

[-45. -45. -45. ... 135. 135. 135.]
